In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt


### Load train data

In [3]:
zip_file_path1 = '../../extracted_data/updated_trainingData1.csv'
train_data1 = pd.read_csv(zip_file_path1)

zip_file_path2 = '../../extracted_data/updated_trainingData2.csv'
train_data2 = pd.read_csv(zip_file_path2)

zip_file_path3 = '../../extracted_data/updated_trainingData3.csv'
train_data3 = pd.read_csv(zip_file_path3)

zip_file_path4 = '../../extracted_data/updated_trainingData4.csv'
train_data4 = pd.read_csv(zip_file_path4)

zip_file_path5 = '../../extracted_data/updated_trainingData5.csv'
train_data5 = pd.read_csv(zip_file_path5)

In [5]:
print(train_data1.shape)
print(train_data2.shape)
print(train_data3.shape)
print(train_data4.shape)
print(train_data5.shape)

(10000, 16800)
(10000, 16800)
(10000, 16800)
(10000, 16800)
(11700, 16800)


In [7]:
train_data = pd.concat([train_data1, train_data2, train_data3, train_data4, train_data5], axis=0)

# Reset index, if necessary, to avoid duplicate indices
train_data.reset_index(drop=True, inplace=True)
print("Combined DataFrame shape:", train_data.shape)

Combined DataFrame shape: (51700, 16800)


In [9]:
train_data.head()

,AN311_value_1,AN311_value_2,AN311_value_3,AN311_value_4,AN311_value_5,AN311_value_6,AN311_value_7,AN311_value_8,AN311_value_9,AN311_value_10,...,V_value_591,V_value_592,V_value_593,V_value_594,V_value_595,V_value_596,V_value_597,V_value_598,V_value_599,V_value_600
0,3.2,3.2,3.2,3.2,3.2,3.2,3.2,3.2,3.2,3.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.7,3.6,3.6,3.6,3.6,3.5,3.5,3.5,3.5,3.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.4,3.4,3.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.2,3.2,3.2,3.3,3.3,3.4,3.4,3.5,3.5,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.4,3.4,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
train_data.to_csv('../../extracted_data/training_data_full_with_columns.csv', index=False)

In [13]:
train_labels1_path = '../../Mining_DataSet/trainingLabels/trainingLabels1.csv'
train_labels2_path = '../../Mining_DataSet/trainingLabels/trainingLabels2.csv'
train_labels3_path = '../../Mining_DataSet/trainingLabels/trainingLabels3.csv'
train_labels4_path = '../../Mining_DataSet/trainingLabels/trainingLabels4.csv'
train_labels5_path = '../../Mining_DataSet/trainingLabels/trainingLabels5.csv'

def load_labels(train_labels_path, train_data):
    train_labels = pd.read_csv(train_labels_path, header = None)
    train_labels.columns = ['MM263', 'MM264', 'MM256']
    print(train_labels.shape)
    return train_labels

train_labels1 = load_labels(train_labels1_path, train_data1)
train_labels2 = load_labels(train_labels2_path, train_data2)
train_labels3 = load_labels(train_labels3_path, train_data3)
train_labels4 = load_labels(train_labels4_path, train_data4)
train_labels5 = load_labels(train_labels5_path, train_data5)

train_labels = pd.concat([train_labels1, train_labels2, train_labels3, train_labels4, train_labels5], axis=0)
train_labels.shape

(10000, 3)
(10000, 3)
(10000, 3)
(10000, 3)
(11700, 3)


(51700, 3)

In [15]:
train_labels.head()

,MM263,MM264,MM256
0,normal,normal,normal
1,normal,normal,normal
2,normal,normal,normal
3,normal,normal,normal
4,normal,normal,normal


In [17]:
train_labels.to_csv('../../extracted_data/train_labels_full_with_columns.csv', index=False)

### Load Test data

In [19]:
test_data = pd.read_csv("../../Mining_DataSet/testData.csv", header=None)

In [21]:
test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,16790,16791,16792,16793,16794,16795,16796,16797,16798,16799
0,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.6,3.5,3.5,3.5,3.5,3.5,3.5,3.2,3.2,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.2,4.2,4.2,4.2,4.3,4.3,4.3,4.3,4.3,4.3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,3.1,3.1,3.2,3.2,3.2,3.2,3.2,3.2,3.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.7,3.7,3.7,3.7,3.7,3.7,3.7,3.7,3.8,3.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# Path to the .txt file containing the column names
column_names_file_path = '../../Mining_DataSet/column_names.txt'

# Read the column names from the .txt file
with open(column_names_file_path, 'r') as file:
    column_names = file.read().strip().split()

test_data.columns = column_names
test_data.head()

,AN311_value_1,AN311_value_2,AN311_value_3,AN311_value_4,AN311_value_5,AN311_value_6,AN311_value_7,AN311_value_8,AN311_value_9,AN311_value_10,...,V_value_591,V_value_592,V_value_593,V_value_594,V_value_595,V_value_596,V_value_597,V_value_598,V_value_599,V_value_600
0,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.6,3.5,3.5,3.5,3.5,3.5,3.5,3.2,3.2,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.2,4.2,4.2,4.2,4.3,4.3,4.3,4.3,4.3,4.3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,3.1,3.1,3.2,3.2,3.2,3.2,3.2,3.2,3.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.7,3.7,3.7,3.7,3.7,3.7,3.7,3.7,3.8,3.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
test_data.to_csv('../../extracted_data/test_data_full_with_columns.csv', index=False)

In [27]:
test_labels = pd.read_csv("../../Mining_DataSet/testLabels/testLabels.csv", header = None)
test_labels.columns = ['MM263', 'MM264', 'MM256']
print(test_labels.shape)
print(test_labels.head())

(5076, 3)
    MM263   MM264    MM256
0  normal  normal  warning
1  normal  normal   normal
2  normal  normal   normal
3  normal  normal   normal
4  normal  normal   normal


In [29]:
test_labels.to_csv('../../extracted_data/test_labels_full_with_columns.csv', index=False)

In [ ]:
# label_mapping = {'normal': 0, 'warning': 1}
# test_labels['MM263'] = test_labels['MM263'].replace(label_mapping).astype(int)
# test_labels['MM264'] = test_labels['MM264'].replace(label_mapping).astype(int)
# test_labels['MM256'] = test_labels['MM256'].replace(label_mapping).astype(int)
# test_labels.head()

In [31]:
# Separate features and target
X_train = train_data
y_train = train_labels['MM263']

X_test = test_data
y_test = test_labels['MM263']

In [33]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(51700, 16800)
(51700,)
(5076, 16800)
(5076,)


In [35]:
counts = train_labels['MM263'].value_counts()  # This gives you a count of each unique value in the column

# Calculate the probability of 'warning'
if 'warning' in counts:
    warning_probability = counts['warning'] / counts.sum()
else:
    warning_probability = 0  # In case there are no 'warning' labels

print("Probability of 'warning':", warning_probability)


Probability of 'warning': 0.008994197292069633


In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, classification_report, confusion_matrix

def trainRF(n):
    rf_classifier = RandomForestClassifier(n_estimators=n, random_state=42, class_weight='balanced')
    rf_classifier.fit(X_train, y_train)
    return rf_classifier



In [39]:
rf_classifier = trainRF(100)

In [41]:
rf_probabilities = rf_classifier.predict_proba(X_test)

In [43]:
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test, rf_probabilities[:, 1])
print("AUC with Random Forest using Probabilities:", auc)

AUC with Random Forest using Probabilities: 0.7950976503255011


In [45]:
# Threshold derived from training data's class distribution
warning_threshold = warning_probability  # The variable computed earlier

# Assuming the 'warning' class is the second column in the probabilities
predicted_labels = np.where(rf_probabilities[:, 1] >= warning_threshold, 'warning', 'normal')

# Check the shape of predicted_labels to ensure it is one-dimensional
print(predicted_labels.shape)




(5076,)


In [47]:
# Classification report and confusion matrix for the new predictions
print(classification_report(y_test, predicted_labels))
print("Confusion Matrix:\n", confusion_matrix(y_test, predicted_labels))

              precision    recall  f1-score   support

      normal       1.00      0.71      0.83      5042
     warning       0.02      0.85      0.04        34

    accuracy                           0.71      5076
   macro avg       0.51      0.78      0.43      5076
weighted avg       0.99      0.71      0.82      5076

Confusion Matrix:
 [[3568 1474]
 [   5   29]]


In [61]:

def trainRF(nestimators,depth ):
    rf_classifier = RandomForestClassifier(
        n_estimators=nestimators,
        max_depth=depth,
        # min_samples_split=50,
        # min_samples_leaf=20,
        # max_features='sqrt',
        random_state=42,
        class_weight='balanced'
    )
    rf_classifier.fit(X_train, y_train)
    rf_probabilities = rf_classifier.predict_proba(X_test)
    auc = roc_auc_score(y_test, rf_probabilities[:, 1])
    print("AUC with combined settings:", auc)

    # Threshold derived from training data's class distribution
    warning_threshold = warning_probability  # The variable computed earlier
    
    # Assuming the 'warning' class is the second column in the probabilities
    predicted_labels = np.where(rf_probabilities[:, 1] >= warning_threshold, 'warning', 'normal')
    
    print(classification_report(y_test, predicted_labels))
    print("Confusion Matrix:\n", confusion_matrix(y_test, predicted_labels))
    # return rf_probabilities


In [55]:
from sklearn.metrics import roc_auc_score

rf_probabilities = trainRF(50, 100)


# Threshold derived from training data's class distribution
warning_threshold = warning_probability  # The variable computed earlier

# Assuming the 'warning' class is the second column in the probabilities
predicted_labels = np.where(rf_probabilities[:, 1] >= warning_threshold, 'warning', 'normal')

print(classification_report(y_test, predicted_labels))
print("Confusion Matrix:\n", confusion_matrix(y_test, predicted_labels))




AUC with combined settings: 0.7699646498821662
              precision    recall  f1-score   support

      normal       1.00      0.75      0.86      5042
     warning       0.02      0.79      0.04        34

    accuracy                           0.75      5076
   macro avg       0.51      0.77      0.45      5076
weighted avg       0.99      0.75      0.85      5076

Confusion Matrix:
 [[3792 1250]
 [   7   27]]


In [57]:
from sklearn.metrics import roc_auc_score

rf_probabilities = trainRF(200, 200)


# Threshold derived from training data's class distribution
warning_threshold = warning_probability  # The variable computed earlier

# Assuming the 'warning' class is the second column in the probabilities
predicted_labels = np.where(rf_probabilities[:, 1] >= warning_threshold, 'warning', 'normal')

print(classification_report(y_test, predicted_labels))
print("Confusion Matrix:\n", confusion_matrix(y_test, predicted_labels))




AUC with combined settings: 0.7689583965279885
              precision    recall  f1-score   support

      normal       1.00      0.74      0.85      5042
     warning       0.02      0.82      0.04        34

    accuracy                           0.74      5076
   macro avg       0.51      0.78      0.44      5076
weighted avg       0.99      0.74      0.84      5076

Confusion Matrix:
 [[3720 1322]
 [   6   28]]


In [59]:
from sklearn.metrics import roc_auc_score

rf_probabilities = trainRF(80, 100)


# Threshold derived from training data's class distribution
warning_threshold = warning_probability  # The variable computed earlier

# Assuming the 'warning' class is the second column in the probabilities
predicted_labels = np.where(rf_probabilities[:, 1] >= warning_threshold, 'warning', 'normal')

print(classification_report(y_test, predicted_labels))
print("Confusion Matrix:\n", confusion_matrix(y_test, predicted_labels))




AUC with combined settings: 0.7782917609725365
              precision    recall  f1-score   support

      normal       1.00      0.72      0.84      5042
     warning       0.02      0.82      0.04        34

    accuracy                           0.72      5076
   macro avg       0.51      0.77      0.44      5076
weighted avg       0.99      0.72      0.83      5076

Confusion Matrix:
 [[3645 1397]
 [   6   28]]


In [63]:
trainRF(70, 150)

AUC with combined settings: 0.7902938843129477
              precision    recall  f1-score   support

      normal       1.00      0.74      0.85      5042
     warning       0.02      0.85      0.04        34

    accuracy                           0.74      5076
   macro avg       0.51      0.79      0.44      5076
weighted avg       0.99      0.74      0.84      5076

Confusion Matrix:
 [[3710 1332]
 [   5   29]]


In [65]:
trainRF(70, 200)

AUC with combined settings: 0.7902938843129477
              precision    recall  f1-score   support

      normal       1.00      0.74      0.85      5042
     warning       0.02      0.85      0.04        34

    accuracy                           0.74      5076
   macro avg       0.51      0.79      0.44      5076
weighted avg       0.99      0.74      0.84      5076

Confusion Matrix:
 [[3710 1332]
 [   5   29]]


In [67]:
trainRF(60, 500)

AUC with combined settings: 0.7969409898032993
              precision    recall  f1-score   support

      normal       1.00      0.74      0.85      5042
     warning       0.02      0.85      0.04        34

    accuracy                           0.74      5076
   macro avg       0.51      0.80      0.45      5076
weighted avg       0.99      0.74      0.85      5076

Confusion Matrix:
 [[3743 1299]
 [   5   29]]


In [69]:
trainRF(70, 1000)

AUC with combined settings: 0.7902938843129477
              precision    recall  f1-score   support

      normal       1.00      0.74      0.85      5042
     warning       0.02      0.85      0.04        34

    accuracy                           0.74      5076
   macro avg       0.51      0.79      0.44      5076
weighted avg       0.99      0.74      0.84      5076

Confusion Matrix:
 [[3710 1332]
 [   5   29]]


In [70]:
trainRF(100, 1000)

AUC with combined settings: 0.7950976503255011
              precision    recall  f1-score   support

      normal       1.00      0.71      0.83      5042
     warning       0.02      0.85      0.04        34

    accuracy                           0.71      5076
   macro avg       0.51      0.78      0.43      5076
weighted avg       0.99      0.71      0.82      5076

Confusion Matrix:
 [[3568 1474]
 [   5   29]]
